In [ ]:
import Pkg
Pkg.activate("..")

In [ ]:
using Test
using PyCall
using Knet
using Debugger: @enter, @bp, @run
using CUDA

if CUDA.functional()
	atype=KnetArray{Float32}
else	
	atype=Array{Float32}
end
cputype=Array{Float32}

In [ ]:
check(v) = any(isnan.(cputype(v))) || any(cputype(v).==Inf)

In [ ]:
include("datasets/sequence.jl")
include("models/common.jl")
include("models/transformers.jl")
include("models/vqvae.jl")
include("setup.jl")

In [ ]:
losssum(prediction) = mean(prediction[2] + prediction[3] + prediction[4])

In [ ]:
model = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_2.jld2", "prev_model");

In [ ]:
value(model.model.codebook.embedding)

# Batch and nanification

In [ ]:
nan_model = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_2.jld2", "model");

In [ ]:
value(nan_model.model.codebook.embedding)

In [ ]:
check(value(nan_model.model.codebook.ema_w))

# batch

In [ ]:
batch = Knet.load("/home/mehmeteneserciyes/logs_julia/hopper-medium-replay-v2/T-1-42/nan_model_2.jld2", "batch");

In [ ]:
joined_inputs, targets, mask, terminals = atype(batch[1]), atype(batch[2]), atype(batch[3]), atype(batch[4]);

In [ ]:
rep, rep_loss, _, commit_loss = nan_model(joined_inputs, targets, mask, terminals)

# step through second

In [ ]:
v = model;

In [ ]:
pred_trajectory = reshape(reconstructed[1:end-1, :, :], (joined_dimension, t, b))

In [ ]:
pred_terminals = reshape(reconstructed[end, :, :], 1,size(reconstructed)[2:end]...)

In [ ]:
weights = cat(
            atype(ones(2)) .* v.position_weight,
            atype(ones(v.observation_dim - 2)),
            atype(ones(v.action_dim)) .* v.action_weight,
            atype(ones(1)) .* v.reward_weight,
            atype(ones(1)) .* v.value_weight,
            dims=1
        )

In [ ]:
mse = mse_loss(pred_trajectory, joined_inputs, reduction="none") .* reshape(weights, (:, 1, 1))

In [ ]:
function binary_cross_entropy(probs,labels; reduction="mean")
    log_pos = atype(clamp.(cputype(log.(probs)), -100.0f0, Inf))
    log_neg = atype(clamp.(cputype(log.(1 .- probs)), -100.0f0, Inf))
    loss = -(labels .* log_pos .+ (1 .- labels) .* log_neg)
    if reduction == "mean"
        mean(loss)
    elseif reduction == "sum"
        sum(loss)
    elseif reduction == "none"
        return loss
    end
end

In [ ]:
check(mse)

In [ ]:
cross_entropy = binary_cross_entropy(pred_terminals, atype(clamp.(convert.(Float32, cputype(terminals)),0.0f0, 1.0f0)))

In [ ]:
labels=atype(clamp.(convert.(Float32, cputype(terminals)),0.0f0, 1.0f0));

In [ ]:
labels .* log.(pred_terminals)

In [ ]:
x = (1 .- labels) .* log.(1 .- pred_terminals);

In [ ]:
pred_terminals[1,7,114]

In [ ]:
?clamp

In [ ]:
log_clamp(x) = log(x) < -100.0f0 ? -100.0f0 : log(x)

In [ ]:
clamp.(pred_terminals, -100.0f0, Inf)

In [ ]:
log_clamp.(1 .- pred_terminals)

In [ ]:
findall(x->isnan(x), cputype(x))

# step through

In [ ]:
v = model;

In [ ]:
joined_dimension, t, b = size(joined_inputs)
padded = repeat_broadcast(v.padding_vector, 1, t, b)

if !(terminals === nothing)
    terminal_mask = repeat_broadcast(deepcopy(1 .- terminals), size(joined_inputs, 1), 1, 1)
    joined_inputs = joined_inputs .* terminal_mask .+ padded .* (1 .- terminal_mask)
end
state = joined_inputs[1:v.observation_dim, 1, :]

In [ ]:
joined_inputs = cat(joined_inputs, terminals, dims=1);
state = state;

In [ ]:
v = model.model;

In [ ]:
check(x_emb)

In [ ]:
token_embeddings = v.embed(joined_inputs)
position_embeddings = v.pos_emb[:, 1:t, :] 
x_emb = v.drop(token_embeddings .+ position_embeddings)

In [ ]:
check(x_emb)

In [ ]:
x_enc = v.encoder(x_emb)

# Attention

In [ ]:
c=v.encoder.layers[1].attn;
x=ln1_out;

In [ ]:
C, T, B = size(ln1_out)

In [ ]:
k = permutedims(reshape(c.key(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
q = permutedims(reshape(c.query(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
v = permutedims(reshape(c.value(x), (C ÷ c.n_head, c.n_head, T, B)), (1, 3, 2, 4));

In [ ]:
att = bmm(permutedims(k, (2,1,3,4)), q) .* Float32(1 / sqrt(size(k, 1)));

In [ ]:
att_masked = att .+ repeat_broadcast(atype((c.mask[1:T,1:T] .== 0) * Float32(-Inf)), 1,1,size(att)[end-1:end]...);

In [ ]:
function softmax(w; dims)
    w = w .- maximum(w, dims=dims)
    probs = exp.(w)
    return probs ./ sum(probs, dims=dims)
end

In [ ]:
att_softmax = softmax(att_masked, dims=1)

In [ ]:
att_masked[:,8,1,1]

In [ ]:
att_softmax[:,8,1,1]

# Rest

In [ ]:
x_pool = permutedims(v.latent_pooling(permutedims(x_enc, (2, 1, 3))), (2,1,3))

In [ ]:
x = v.cast_embed(x_pool)

In [ ]:
check(x)

In [ ]:
findall(x->x==Inf, cputype(x))

In [ ]:
findall(x->x==Inf, cputype(x))